In [6]:
import os
import numpy as np

In [7]:
# Load Glove Embedding
gloveFile = "glove.6B/glove.6B.50d.txt"
glove_model = {}

with open(gloveFile, 'r') as f:
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        glove_model[word] = embedding
    print "Done.",len(glove_model)," words loaded!"  

print glove_model['unk']

Done. 400000  words loaded!
[ -7.91490000e-01   8.66170000e-01   1.19980000e-01   9.22870000e-04
   2.77600000e-01  -4.91850000e-01   5.01950000e-01   6.07920000e-04
  -2.58450000e-01   1.78650000e-01   2.53500000e-01   7.65720000e-01
   5.06640000e-01   4.02500000e-01  -2.13880000e-03  -2.83970000e-01
  -5.03240000e-01   3.04490000e-01   5.17790000e-01   1.50900000e-02
  -3.50310000e-01  -1.12780000e+00   3.32530000e-01  -3.52500000e-01
   4.13260000e-02   1.08630000e+00   3.39100000e-02   3.35640000e-01
   4.97450000e-01  -7.01310000e-02  -1.21920000e+00  -4.85120000e-01
  -3.85120000e-02  -1.35540000e-01  -1.63800000e-01   5.23210000e-01
  -3.13180000e-01  -1.65500000e-01   1.19090000e-01  -1.51150000e-01
  -1.56210000e-01  -6.26550000e-01  -6.23360000e-01  -4.21500000e-01
   4.18730000e-01  -9.24720000e-01   1.10490000e+00  -2.99960000e-01
  -6.30030000e-03   3.95400000e-01]


In [8]:
# Prepare dataset for NLTK HMM Tagger
# Divide in train and test files [80:20] 

# Directory having content
doc_dir = 'Data/content'

train_file_list = []
test_file_list = []

for f in os.listdir(doc_dir):
    #Random Sampling
    if np.random.uniform(0,1)< 0.8:
        train_file_list.append(f)
    else:
        test_file_list.append(f)

print len(train_file_list)
print len(test_file_list)

1043
264


In [9]:
# Directory having tags
tag_dir = 'Data/new_tags'

tagEmbedding = dict()
tagCount = dict()
win_size = 2
word_dim = 50

for f in train_file_list:
    word_file_path = os.path.join(doc_dir, f)
    tag_file_path = os.path.join(tag_dir, f)
    lines_in_word_file = []
    lines_in_tag_file = []
    with open(word_file_path, "rt") as word_file:
        for line in word_file:
            lines_in_word_file.append(line)
    with open(tag_file_path, "rt") as tag_file:
        for line in tag_file:
            lines_in_tag_file.append(line)
    if (len(lines_in_word_file) == len(lines_in_tag_file)) and len(lines_in_word_file) > 0:
        for i in xrange(len(lines_in_word_file)):
            word_in_file = lines_in_word_file[i].split()
            tag_in_file = lines_in_tag_file[i].split()
            length = min(len(word_in_file), len(tag_in_file))
            for j in xrange(length):
                # Get the word embedding of the word
                word_embed = np.zeros(word_dim)
                if (j>(win_size-1)) and (j<length-win_size):
                    for k in xrange(2*win_size + 1):
                        if word_in_file[j-win_size+k] in glove_model:
                            word_embed = np.add(word_embed, glove_model[word_in_file[j-win_size+k]])
                        else:
                            word_embed = np.add(word_embed, glove_model['unk'])  
                # Create embdedding of Tag    
                if tag_in_file[j] in tagEmbedding:
                    tagEmbedding[tag_in_file[j]] = np.add(tagEmbedding[tag_in_file[j]], word_embed)
                    tagCount[tag_in_file[j]] = tagCount[tag_in_file[j]] + 1
                else:
                    tagEmbedding[tag_in_file[j]] = word_embed
                    tagCount[tag_in_file[j]] = 1
                    

In [17]:
for key in tagCount:
    tagEmbedding[key] = tagEmbedding[key]/tagCount[key]
    
print tagCount

print '\nPER_Assoc\n'
print tagEmbedding['PER_Assoc']
print '\nORG_Assoc\n'
print tagEmbedding['ORG_Assoc']
print '\n'
print tagEmbedding['PER_Others']
print '\n'
print tagEmbedding['Victim']
print '\n'
print tagEmbedding['LOC_Event']
print '\n'
print tagEmbedding['O']
print '\n'
print tagEmbedding['ORG_Accused']
print '\n'
print tagEmbedding['PER_Victim']
print '\n'
print tagEmbedding['ORG_Victim']
print '\n'
print tagEmbedding['LOC_Others']
print '\n'
print tagEmbedding['ORG_Others']
print '\n'
print tagEmbedding['PER_Accused']
print '\n'
print tagEmbedding['LOC_Accused']
print '\n'
print tagEmbedding['LOC_Assoc']
print '\n'
print tagEmbedding['LOC_Victim']
print '\n'
print tagEmbedding['Event']
print '\n'
print tagEmbedding['Location']
                    

{'PER_Assoc': 226, 'ORG_Assoc': 47, 'PER_Others': 7525, 'Victim': 29, 'LOC_Event': 5355, 'Accused': 244, 'O': 355257, 'ORG_Accused': 2421, 'PER_Victim': 1078, 'ORG_Victim': 322, 'LOC_Others': 7282, 'ORG_Others': 9471, 'PER_Accused': 3483, 'LOC_Accused': 645, 'LOC_Assoc': 337, 'LOC_Victim': 195, 'Event': 336, 'Location': 555}

PER_Assoc

[-0.11802384  1.56611233  0.19118969  0.06608645  1.77025962  0.16128241
 -0.69369645 -0.26833159 -1.0402942  -0.34265823  0.57402095  0.9075128
 -0.66454502  0.05287937  1.26286336 -0.54840358 -1.11148607  0.44877647
 -0.83546898 -0.13815065  0.07163313 -0.89433765  1.31824573 -0.68930918
 -0.45604865 -3.92301763 -0.27812348  0.13703477  0.69541167 -0.59457385
  8.48005095 -1.11871456 -1.01844732 -1.53299165  0.41149979  0.95987922
 -0.09956742 -0.13719224  0.42631865  0.05255309 -0.73401167 -0.20772485
 -0.73838156 -0.90738872  0.81527577 -1.62462992  0.52851504 -1.05803801
  0.16332661 -0.16948711]

ORG_Assoc

[-0.4007283   1.59685749  0.20116627  0.